# Google ADK Interactive Testing Notebook

This notebook allows you to interactively test Google ADK agents.

In [ ]:
# Setup and imports
import os
import sys
from dotenv import load_dotenv
import asyncio
from IPython.display import display, Markdown

# Load environment variables
load_dotenv()

# Configure API key
google_api_key = os.getenv('GOOGLE_API_KEY')
if not google_api_key:
    print("❌ Error: GOOGLE_API_KEY not found in environment")
else:
    os.environ['GOOGLE_API_KEY'] = google_api_key
    print("✅ GOOGLE_API_KEY loaded successfully")

In [ ]:
# Import ADK components
from google.adk import Runner
from google.adk.agents import Agent, SequentialAgent, ParallelAgent, LlmAgent
from google.adk.sessions import InMemorySessionService
from google.genai import types
from pydantic import BaseModel, Field

## 1. Simple Chat Agent

In [ ]:
# Create a simple chat agent
simple_agent = Agent(
    name="assistant",
    model="gemini-2.0-flash",
    description="A helpful AI assistant",
    instruction="""You are a helpful, friendly AI assistant. 
    Provide clear and concise responses."""
)

# Create session for simple agent
session_service = InMemorySessionService()
simple_runner = Runner(
    agent=simple_agent,
    app_name="notebook_test",
    session_service=session_service
)

simple_session = session_service.create_session(
    app_name="notebook_test",
    user_id="notebook_user",
    session_id="simple_chat"
)

print("✅ Simple chat agent ready!")

In [ ]:
# Chat with simple agent
async def chat_simple(message: str):
    content = types.Content(role='user', parts=[types.Part(text=message)])
    
    response_text = ""
    async for event in simple_runner.run_async(
        user_id="notebook_user",
        session_id=simple_session.id,
        new_message=content
    ):
        if event.is_final_response():
            if event.content and event.content.parts:
                response_text = '\n'.join([p.text for p in event.content.parts if p.text])
    
    return response_text

# Example usage
response = await chat_simple("Hello! Can you explain what Google ADK is?")
display(Markdown(f"**Response:** {response}"))

## 2. Sequential Agent (Research + Writing)

In [ ]:
# Create sequential agent with research and writing sub-agents
research_agent = Agent(
    name="researcher",
    model="gemini-2.0-flash",
    description="Research specialist",
    instruction="""You are a research specialist. 
    Provide key facts and insights about the given topic."""
)

writer_agent = Agent(
    name="writer",
    model="gemini-2.0-flash",
    description="Content writer",
    instruction="""You are a creative writer. 
    Take the research and craft it into engaging content."""
)

sequential_agent = SequentialAgent(
    name="research_writer",
    sub_agents=[research_agent, writer_agent],
    description="Research and writing team"
)

# Create runner for sequential agent
seq_runner = Runner(
    agent=sequential_agent,
    app_name="notebook_test",
    session_service=session_service
)

seq_session = session_service.create_session(
    app_name="notebook_test",
    user_id="notebook_user",
    session_id="sequential_chat"
)

print("✅ Sequential agent ready!")

In [ ]:
# Use sequential agent
async def research_and_write(topic: str):
    content = types.Content(role='user', parts=[types.Part(text=topic)])
    
    response_text = ""
    async for event in seq_runner.run_async(
        user_id="notebook_user",
        session_id=seq_session.id,
        new_message=content
    ):
        if event.is_final_response():
            if event.content and event.content.parts:
                response_text = '\n'.join([p.text for p in event.content.parts if p.text])
    
    return response_text

# Example usage
article = await research_and_write("The future of renewable energy")
display(Markdown(f"**Article:**\n\n{article}"))

## 3. Structured Output Agent

In [ ]:
# Define structured output schema
class AnalysisResult(BaseModel):
    summary: str = Field(description="Brief summary")
    key_points: list[str] = Field(description="Main points")
    pros: list[str] = Field(description="Advantages")
    cons: list[str] = Field(description="Disadvantages")
    recommendation: str = Field(description="Final recommendation")

# Create structured output agent
structured_agent = LlmAgent(
    name="analyst",
    model="gemini-2.0-flash",
    instruction="""Analyze topics and provide structured insights.
    Always provide balanced analysis.""",
    output_schema=AnalysisResult,
    output_key="analysis"
)

# Create runner
struct_runner = Runner(
    agent=structured_agent,
    app_name="notebook_test",
    session_service=session_service
)

struct_session = session_service.create_session(
    app_name="notebook_test",
    user_id="notebook_user",
    session_id="structured_chat"
)

print("✅ Structured agent ready!")

In [ ]:
# Analyze with structured output
import json

async def analyze_topic(topic: str):
    content = types.Content(role='user', parts=[types.Part(text=f"Analyze: {topic}")])
    
    response_text = ""
    async for event in struct_runner.run_async(
        user_id="notebook_user",
        session_id=struct_session.id,
        new_message=content
    ):
        if event.is_final_response():
            if event.content and event.content.parts:
                response_text = '\n'.join([p.text for p in event.content.parts if p.text])
    
    # Parse JSON response
    try:
        analysis = json.loads(response_text)
        return analysis
    except:
        return {"error": "Could not parse response", "raw": response_text}

# Example usage
analysis = await analyze_topic("Working from home vs office work")

# Display formatted results
if "error" not in analysis:
    display(Markdown(f"### Summary\n{analysis['summary']}"))
    display(Markdown(f"\n### Key Points\n" + "\n".join([f"- {p}" for p in analysis['key_points']])))
    display(Markdown(f"\n### Pros\n" + "\n".join([f"✅ {p}" for p in analysis['pros']])))
    display(Markdown(f"\n### Cons\n" + "\n".join([f"❌ {c}" for c in analysis['cons']])))
    display(Markdown(f"\n### Recommendation\n{analysis['recommendation']}"))
else:
    print(analysis)

## 4. Interactive Chat Cell

Use this cell to have ongoing conversations with any agent:

In [ ]:
# Interactive chat function
def create_chat_widget(agent_runner, session):
    from ipywidgets import widgets, Layout
    from IPython.display import display, clear_output
    
    # Create widgets
    chat_output = widgets.Output()
    message_input = widgets.Text(
        placeholder='Type your message here...',
        layout=Layout(width='70%')
    )
    send_button = widgets.Button(
        description='Send',
        button_style='primary',
        layout=Layout(width='20%')
    )
    clear_button = widgets.Button(
        description='Clear',
        button_style='warning',
        layout=Layout(width='10%')
    )
    
    # Chat history
    chat_history = []
    
    async def send_message(_):
        message = message_input.value
        if not message:
            return
        
        # Clear input
        message_input.value = ''
        
        # Add to history
        chat_history.append(f"**You:** {message}")
        
        # Get response
        content = types.Content(role='user', parts=[types.Part(text=message)])
        
        response_text = ""
        async for event in agent_runner.run_async(
            user_id="notebook_user",
            session_id=session.id,
            new_message=content
        ):
            if event.is_final_response():
                if event.content and event.content.parts:
                    response_text = '\n'.join([p.text for p in event.content.parts if p.text])
        
        chat_history.append(f"**Agent:** {response_text}")
        
        # Update display
        with chat_output:
            clear_output()
            for msg in chat_history:
                display(Markdown(msg + "\n\n---\n"))
    
    def clear_chat(_):
        chat_history.clear()
        with chat_output:
            clear_output()
    
    # Connect buttons
    send_button.on_click(lambda _: asyncio.create_task(send_message(_)))
    message_input.on_submit(lambda _: asyncio.create_task(send_message(_)))
    clear_button.on_click(clear_chat)
    
    # Display widgets
    display(chat_output)
    display(widgets.HBox([message_input, send_button, clear_button]))

# Create chat widget for simple agent
print("💬 Chat with the Simple Agent:")
create_chat_widget(simple_runner, simple_session)

## 5. Test Your Own Agent

Create and test your own custom agent:

In [ ]:
# Create your own agent here
my_agent = Agent(
    name="my_custom_agent",
    model="gemini-2.0-flash",
    description="My custom agent",
    instruction="""[Replace this with your custom instructions]
    
    Example: You are an expert in [domain]. 
    Help users with [specific tasks].
    Always [specific behavior]."""
)

# Create runner
my_runner = Runner(
    agent=my_agent,
    app_name="notebook_test",
    session_service=session_service
)

my_session = session_service.create_session(
    app_name="notebook_test",
    user_id="notebook_user",
    session_id="my_agent_chat"
)

# Test it
async def test_my_agent(message: str):
    content = types.Content(role='user', parts=[types.Part(text=message)])
    
    response_text = ""
    async for event in my_runner.run_async(
        user_id="notebook_user",
        session_id=my_session.id,
        new_message=content
    ):
        if event.is_final_response():
            if event.content and event.content.parts:
                response_text = '\n'.join([p.text for p in event.content.parts if p.text])
    
    return response_text

# Test your agent
response = await test_my_agent("Hello! What can you help me with?")
display(Markdown(f"**Response:** {response}"))